<a href="https://colab.research.google.com/github/SeongYun1/CodeLab/blob/main/QM_fewshot_rev2_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 60000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np

from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from tensorflow.keras.utils import to_categorical

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

KeyboardInterrupt: 

In [ ]:
df = pd.read_excel("dataset02.xlsx")
df.head(1)
df.shape

,호선,H.C/Close,진행상태,부서진행상태,QM확인,SCAS_CNO,코멘트번호,ORI_CMT_NO,제목,문제내용,문제내용(ICON),문제내용(번역),문제내용(번역 ICON),파일,진행상태2,처리부서,처리과,처리부서(후처리),담당자,담당자 선택,확정자,이엔티/부서,이엔티/과,REPLY 상태,회신내용,Maker 조치예정일,Maker업체명,Maker작업자,Maker연락처,Maker eMail,Maker 선택,완료예정일,원인부서,원인과,구역,BLOCK,LOCATION,결함코드,원인,발행자이름,검사원성명,후속호선 반영여부,개정도유무,HHI COMPLETED일자,종료일자,종료자1,S/T(ref),S/T,지연일,접수지연일,최근회신일,발행일,등록일,접수일,G/T(ref),G/T,AS이관일,QM확인완료일,최초회신일,회신일,회신건수,3일이내 회신건수,처리일,처리건수,3일이내 처리건수,최종변경자성명,소요공수,실패비용,중요도,지연보류 해제,지연보류 해제일,검사제한여부,ORDER NO.,DESC.,주관부서,검사결과,처리부서 미조치,LQC 소속,LQC 이름,Maker A/S,자재입고,S/T3,G/T4
0,3040,Indeterminate,CLOSED,QM확인완료,Unchecked,NaN,C-3040-B-AB-00001,NaN,Bunker specification using the sea trial to be submitted to the class.,Bunker specification using the sea trial to be submitted to the class.,Bunker specification using the sea trial to be submitted to the class.,NaN,NaN,NaN,CLOSED,시운전부,기관1과(C853-000),Unchecked,이번하,Indeterminate,NaN,NaN,NaN,QM Confirmed(QM확인완료),금일 선주 사인 받았습니다. 종결처리 부탁드립니다.\n(첨부 사진 전송 완료),NaN,NaN,NaN,NaN,NaN,Unchecked,2019-03-22,시운전부(C850),기관1과(C853-000),NaN,해당없음,NaN,JEE,"JEE:작업예정(자재미입고, 싯점대기)",박정준(J.J. PARK),김태진,Unchecked,NaN,NaN,2019-03-13,김태진,2019-03-04,2019-02-07,"1,094",1.0,2019-03-13,2019-03-11,2019-03-11,2019-03-12,NaN,NaN,NaN,2019-03-13,2019-03-12,NaN,1,1,2.0,1,1,김태진,NaN,"171,456.0",중,Unchecked,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,Indeterminate,Indeterminate,Indeterminate,Indeterminate


(47741, 83)

In [ ]:
df = df[["호선", "코멘트번호", "제목", "문제내용", "처리부서", "처리과", "회신내용", "구역", "LOCATION", "BLOCK"]]
df.shape

(47741, 10)

In [ ]:
text_to_drop = ['이관', "재지정", "재 지정"]
for drop_word in text_to_drop:
    df = df[~df['회신내용'].str.contains(drop_word, regex=True, na=False)]
df.shape

(46145, 10)

In [ ]:
def contains_word(main_string, target_word):
    return target_word in main_string

기준직능 = ["선장", "기장","전장", "선실", "엔진", "기관", "기계", "도장", "건조", "시운전","발판", "보냉", "공사"]
         #"의장", "설계","선행",

def 직능구하기(string):
    global 기준직능
    result = "일반"
    for 직능 in 기준직능:
        if contains_word(str(string), 직능):
            result = 직능
        else:
            pass
    return result

df["직능"] = df["처리과"].apply(직능구하기)

df["직능"].fillna("일반", inplace=True)
df["직능"].value_counts()

직능
일반     15135
기장      7894
전장      7537
선장      3889
선실      3847
공사      2219
도장      1568
엔진      1564
기계       937
기관       636
시운전      500
건조       302
발판        70
보냉        47
Name: count, dtype: int64

In [ ]:
df['직능'] = df['직능'].str.replace("보냉", '도장', regex=True)
df['직능'] = df['직능'].str.replace("발판", '도장', regex=True)

In [ ]:
df.shape
df = df[df["직능"]!="일반"]
df.shape

(46145, 11)

(31010, 11)

In [ ]:
df = df.drop_duplicates(subset='코멘트번호', keep='first')
df.shape

(22668, 11)

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14043494939643941341
 xla_global_id: -1]

In [ ]:
import functools # not required, but helps in production
def unpack_df_columns(func):

    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):

        series = args[0]
        return func(*series.values)

    return _unpack_df_columns

def jaccard_simil(a, b):
    intersection_cardinality = len(set.intersection(*[set(str(a)), set(str(b))]))
    union_cardinality = len(set.union(*[set(str(a)), set(str(b))]))
    similar = intersection_cardinality / float(union_cardinality)
    return similar

@unpack_df_columns
def 문장병합(제목, 문제내용):
    '''
    자카드유사도가 일정 수준 이상이면 제목과 문제내용중 길이가 긴 내용만 살리기
    '''
    자카드유사도 = jaccard_simil(제목, 문제내용)
    thresh = 0.9  # 유사도 커트라인

    if 자카드유사도 > thresh:
        if len(str(제목)) > len(str(문제내용)):
            return 문제내용
        else:
            return 제목
    else:
        return f"{제목} {문제내용}"

In [ ]:
df['comment'] = df[['제목',"문제내용"]].apply(문장병합, axis=1)
df.dropna(subset=["comment"], inplace=True)

In [ ]:
korean_pattern = '[\u3131-\u3163\uac00-\ud7a3]+'
df = df[~df['comment'].str.contains(korean_pattern, regex=True)]

In [ ]:
df1 =df[["comment", "직능"]]
df1.columns = ["text", "label"]
df1 = df1.reset_index(drop=True)
df1.head(2)

,text,label
0,Bunker specification using the sea trial to be submitted to the class.,기관
1,1. The following operational procedures are to be provided.\n'1) Fuel-change over,기장


In [ ]:
new_labels = {'기관':0, '기장':1, '선장':2, '전장':3, '선실':4, '건조':5, '시운전':6, '도장':7, '엔진':8, '기계':9, '공사':10}
df1["label"] = df1["label"].replace(to_replace=new_labels)
df1["label"]

0        0
1        1
2        0
3        1
4        2
        ..
18919    3
18920    4
18921    6
18922    3
18923    2
Name: label, Length: 18924, dtype: int64

In [ ]:
classes = new_labels.keys()
classes = list(classes)
classes

['기관', '기장', '선장', '전장', '선실', '건조', '시운전', '도장', '엔진', '기계', '공사']

In [ ]:
from sklearn.model_selection import train_test_split
df1.shape
train, val= train_test_split(df1, test_size=0.3, random_state=123, shuffle=True, stratify=df1["label"])
train.shape, val.shape

(18924, 2)

((13246, 2), (5678, 2))

In [ ]:
val, test= train_test_split(val, test_size=0.3, random_state=123, shuffle=True, stratify=val["label"])

val.shape, test.shape

((3974, 2), (1704, 2))

In [ ]:
!pip install datasets transformers accelerate setfit

^C


In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
train1 = Dataset.from_pandas(train)
val1 = Dataset.from_pandas(val)
test1  = Dataset.from_pandas(test)

In [ ]:
dataset = DatasetDict()

In [ ]:
dataset["train"] = train1
dataset["validation"] = val1
dataset["test"] = test1

In [ ]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 13246
})

In [ ]:
val_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 3974
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 1704
})

In [ ]:
def get_train_dataset(dataset, N):
    ids = []
    label2count = {}
    train_dataset = dataset['train'].shuffle(seed=41)
    for id, example in enumerate(train_dataset):
        if example['label'] not in label2count:
            label2count[example['label']]=1
        elif label2count[example['label']]>=N:
            continue
        else:
            label2count[example['label']]+=1
        ids.append(id)
    return train_dataset.select(ids)

In [ ]:
N = 5
train_dataset = get_train_dataset(dataset, N)
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 55
})

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments
from sklearn.metrics import classification_report

In [ ]:
model = SetFitModel.from_pretrained("BAAI/bge-base-en-v1.5")

args1 = TrainingArguments(
    batch_size=32,
    num_epochs=1,
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    # load_best_model_at_end=True,
)

trainer1 = Trainer(
    model=model,
    args=args1,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    metric="accuracy",
    column_mapping={"text": "text", "label": "label"}
)

C:\Users\irene\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


KeyboardInterrupt: 

In [ ]:
trainer1.train()

In [ ]:
metrics = trainer1.evaluate()
metrics

In [ ]:
# trainer1.model._save_pretrained(save_directory="./model/")

In [ ]:
preds = model.predict(test_dataset['text'])
preds

In [ ]:
# print(classification_report(test_dataset['label'], preds, target_names=classes, digits=4))

In [ ]:
print(classification_report(test_dataset['label'], preds, digits=4))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'knowledgator/comprehend_it-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import random
import torch
import evaluate
import numpy as np

In [ ]:
accuracy = evaluate.load("accuracy")
accuracy

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
# 샘플 데이터를 10개씩 증강.. (원래 48개인데.. 10문장씩 복사 증강하여 총 데이터가 480개가 됨)

def transform_dataset(dataset, classes, template = '{}'):
   new_dataset = {'sources':[], 'targets': [], 'labels': []}

   texts = dataset['text']
   print(f"texts 앞에 두개: {texts}")
   labels = dataset['label']
   print(f"labels 앞에 두개: {labels}")

   label2count = {}
   for label in labels:
       if label not in label2count:
           label2count[label]=1
       else:
           label2count[label]+=1
   print(f"label2count: {label2count}")

   count = len(labels)
   print(f"count: {count}")

   label2prob = {label:lc/count for label, lc in label2count.items()}
   print(f"label2prob: {label2prob}")

   unique_labels = list(label2prob)
   print(f"unique_labels: {unique_labels}")

   probs = list(label2prob.values())
   print(f"probs: {probs}")

   ids = list(range(len(labels)))
   print(f"ids: {ids}")

   print(f"classes: {classes}")

   for text, label_id in zip(texts, labels):
       label = classes[label_id]
       for i in range(len(classes)-1):
           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(label))
           new_dataset['labels'].append(1.)

       print(f"new_dataset1: {new_dataset}")

       for i in range(len(classes)-1):
           neg_class_ = label
           while neg_class_==label:
               # neg_class_ = random.sample(classes, k=1)[0]
               neg_lbl = np.random.choice(unique_labels, p=probs)
               neg_class_ = classes[neg_lbl]

           new_dataset['sources'].append(text)
           new_dataset['targets'].append(template.format(neg_class_))
           new_dataset['labels'].append(-1.)
       print(f"new_dataset2: {new_dataset}")

   result = Dataset.from_dict(new_dataset)
   print(f"result : {result}")
   return result

In [ ]:
def compute_metrics(eval_pred):
   predictions, labels = eval_pred

   predictions = np.argmax(predictions, axis=1)

   return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
def tokenize_and_align_label(example):
   hypothesis = example['targets']

   seq = example["sources"]+hypothesis

   tokenized_input = tokenizer(seq, truncation=True, max_length=512,
                                                    padding="max_length")

   label = example['labels']
   if label==1.0:
       label = torch.tensor(1)
   elif label==0.0:
       label = torch.tensor(2)
   else:
       label = torch.tensor(0)
   tokenized_input['label'] = label
   return tokenized_input

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DebertaV2TokenizerFast(name_or_path='knowledgator/comprehend_it-base', vocab_size=128000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
dataset = transform_dataset(train_dataset, classes)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_label)

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 990
    })
    test: Dataset({
        features: ['sources', 'targets', 'labels', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 110
    })
})

In [ ]:
training_args = TrainingArguments(
   output_dir='comprehendo',
   learning_rate=3e-5,
   per_device_train_batch_size=8,
   per_device_eval_batch_size=8,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
)

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset['test'],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.227876,0.918182
2,No log,0.094219,0.963636
3,No log,0.048557,0.972727


TrainOutput(global_step=372, training_loss=0.26625844483734457, metrics={'train_runtime': 166.8186, 'train_samples_per_second': 17.804, 'train_steps_per_second': 2.23, 'total_flos': 928807021209600.0, 'train_loss': 0.26625844483734457, 'epoch': 3.0})

In [ ]:
trainer.save_model('comprehender')

In [ ]:
from transformers import pipeline
from sklearn.metrics import classification_report
from tqdm import tqdm

classifier = pipeline("zero-shot-classification",
                     model='comprehender',tokenizer=tokenizer, device=device)

In [ ]:
preds = []
label2idx = {label: id for id, label in enumerate(classes)}

for example in tqdm(test_dataset):
   pred = classifier(example['text'],classes)['labels'][0]
   idx = label2idx[pred]
   preds.append(idx)

print(classification_report(test_dataset['label'], preds,
                                        target_names=classes, digits=4))

100%|██████████████████████████████████████████████████████████████████████████████| 1704/1704 [04:29<00:00,  6.33it/s]

              precision    recall  f1-score   support

          기관     0.0000    0.0000    0.0000        29
          기장     0.2674    0.1015    0.1472       453
          선장     0.1429    0.0124    0.0229       241
          전장     0.3072    0.1144    0.1667       411
          선실     0.0973    0.2667    0.1426       165
          건조     0.0333    0.0667    0.0444        15
         시운전     0.0127    0.1579    0.0234        19
          도장     0.1061    0.3256    0.1600        86
          엔진     0.3975    0.8559    0.5429       111
          기계     0.0204    0.0541    0.0296        37
          공사     0.2667    0.0584    0.0958       137

    accuracy                         0.1626      1704
   macro avg     0.1501    0.1830    0.1251      1704
weighted avg     0.2284    0.1626    0.1488      1704

